In [1]:
!pip3 install -r requirements.txt

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [19]:
import os
import streamlit
import pickle
import time
import tqdm as notebook_tqdm
import langchain
import numpy as np
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains import RetrievalQAWithSourcesChain
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore import InMemoryDocstore
import faiss

In [3]:
from transformers import pipeline
from langchain.llms.base import LLM
from typing import List, Dict, Any

class TransformersLLM(LLM):
    def __init__(self, pipe):
        super().__init__()
        object.__setattr__(self, 'pipe', pipe)

    @property
    def _llm_type(self) -> str:
        return "transformers"

    def _call(self, question: str, context: str) -> str:
        result = self.pipe(question=question, context=context)
        return result['answer']

    def _generate(self, prompts: List[str], contexts: List[str], **kwargs) -> List[Dict[str, Any]]:
        return [{"text": self._call(prompt, context)} for prompt, context in zip(prompts, contexts)]


# Initialize the pipeline
pipe = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
llm = TransformersLLM(pipe)

pipe1 = pipeline("summarization", model = "facebook/bart-large-cnn")
llm1 = TransformersLLM(pipe1)

# Function to generate a single statement combining the information from chunks
def generate_summary_statement(chunks: List[str], question: str, llm: TransformersLLM) -> str:
    # print("Lookin")
    combined_text = " ".join(chunks)
    prompt = f"Summarize the following information to answer the question: {question}\n\n{combined_text}"
    print(f"Combined text:\n{combined_text}\n")  # Debug: Check combined text
    print(f"Question: {question}\n")  # Debug: Check question
    summary = llm._call(prompt, context=combined_text)
    
    return summary

In [4]:
url1= 'https://apnews.com/article/debate-trump-biden-cnn-camp-david-florida-2f67b119f58136b310b625e42fad3d3a'
url2 = 'https://www.nytimes.com/2024/06/23/us/politics/biden-trump-debate-stakes.html'
url3 = 'https://www.yahoo.com/news/biden-trump-presidential-debate-how-to-watch-what-to-know-elections-republican-democrat-cnn-173153159.html'

## Data Loader

In [5]:
data_loader = UnstructuredURLLoader(urls = (url1,url2,url3))

In [6]:
documents = data_loader.load()

## Document Splitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 250,
        chunk_overlap = 50,
        separators = ['\n\n','\n', '.', ' '],
)

In [8]:
docs = text_splitter.split_documents(documents)

## Creating word embeddings

In [9]:
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

texts = [doc.page_content for doc in docs]
embeddings = model.encode(texts)


/Users/khushi/Desktop/Chat_bot/chat_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
embedding_model = HuggingFaceEmbeddings(model_name='paraphrase-MiniLM-L6-v2')


/Users/khushi/Desktop/Chat_bot/chat_env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [11]:
vector_index = FAISS.from_documents(docs,embedding_model)

## Saving the embeddings file for future use

In [14]:
file_name = "vector_index.pkl"
with open(file_name, 'wb') as f:
    pickle.dump(vector_index,f)

In [15]:
if os.path.exists(file_name):
    with open(file_name ,'rb') as f:
        vector_index = pickle.load(f)

In [12]:
retriever = vector_index.as_retriever()


In [13]:
# retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})
        
# # Load QA chain
# qa_chain = load_qa_with_sources_chain(llm, chain_type="stuff")
# qa_with_sources = RetrievalQAWithSourcesChain(combine_documents_chain=qa_chain, retriever=retriever, return_source_documents=True)
# result = qa_with_sources._call({"question": question_prompt, "contexts": docs})



In [14]:
question_prompt = "who are hosting the show" 
langchain.debug = True

In [15]:
docs = retriever.get_relevant_documents(question_prompt)

/Users/khushi/Desktop/Chat_bot/chat_env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [16]:
chunks = [chunk.page_content for chunk in docs]

In [17]:
answer = generate_summary_statement(chunks,question_prompt,llm)

Combined text:
On Saturday, he’s set to host a rally in Philadelphia and deliver a keynote address to a conference of Christian conservatives in Washington. He also has a fundraiser in New Orleans on Monday before going to his Florida estate for meetings. 🗣️ Who’s moderating the debate?

Jake Tapper and Dana Bash, co-hosts of CNN’s Sunday morning show State of the Union, will serve as moderators. Bash, a graduate of George Washington University, anchors CNN’s Inside Politics with Dana Bash and has regularly served as moderator for numerous political town halls and debates — including six presidential primary debates in 2016 and two in 2020. ⌚ When, where and how to watch

The first debate was hosted by CNN at the cable network’s studios in Atlanta and ran with only two commercial breaks.

Yahoo.com will also feature coverage and analysis in real-time from our editorial team.

Question: who are hosting the show



In [18]:
answer

'Saturday, he’s set to host a rally in Philadelphia'

In [20]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


/Users/khushi/Desktop/Chat_bot/chat_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
query = "who are hosting the show"
context = " ".join(chunks)
qa_input = {
    'question': query,
    'context': context
}

result = qa_pipeline(qa_input)
answer = result['answer']

In [22]:
answer

'Jake Tapper and Dana Bash'

In [75]:
embeddings = np.array(embeddings).astype('float32')
d = embeddings.shape[1]  # Dimension of the embeddings
index = faiss.IndexFlatL2(d)
index.add(embeddings)


In [79]:
# Initialize the docstore
docstore = InMemoryDocstore(dict(enumerate(documents)))

# Create FAISS vector store from embeddings
vector_store = FAISS(embeddings, index, docstore, dict(enumerate(documents)))

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [69]:
query = "Biden Lost"
query_embedding = model.encode([query]).astype('float32')

In [70]:
k = 2  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

In [71]:
print(f"Nearest neighbors (indices): {indices}")
print(f"Nearest neighbors (distances): {distances}")

Nearest neighbors (indices): [[46  5]]
Nearest neighbors (distances): [[35.063553 41.40638 ]]


In [72]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = pipe, retriever=index)

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)